In [3]:
import pickle
import glob
import pandas as pd
import numpy as np

import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials


## Load Music Data / Data Collection
***

In [ ]:
artname = pd.read_csv('data/usersha1-artmbid-artname-plays.tsv', sep='\t', header=None, names = ['users', 'artist_id', 'artist_name', 'plays'],
                        usecols = ['users', 'artist_name', 'plays'])
profile = pd.read_csv('data/usersha1-profile.tsv', sep='\t', header=None, names = ['users', 'gender', 'age', 'country', 'sign_up'],
                        usecols = ['users', 'country'])


In [ ]:
#Getting all the spotify_data follwing by numbers
filenames = glob.glob('data/spotify_data*.csv')
l = []
for file in filenames: 
    df = pd.read_csv(file)
    l.append(df)
    
spotify_data = pd.concat(l, axis = 0,ignore_index= True)
spotify_data.head()


In [ ]:
#SpotifyClientCredentials that can be used to authenticate requests like
#Using Spotify API to collect more detailed audio features
client_credentials_manager = SpotifyClientCredentials(client_id="ad1a53bb434c4023a44d59b0c5ac533b",client_secret='e25b1aa4183d47b7abb9c0494bb05696')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)



In [ ]:
trackid = list(set(spotify_data.trackid))
batchsize = 50

def spotify_info(trackid,batchsize):
    features = []
    popularity = []
    uri = []
    artists_uri = []
    none_count = 0
    none_count2 = 0
    for i in range(0,len(trackid),batchsize):
        batch = trackid[i:i+batchsize]
        track_results = sp.tracks(batch)
        features_results = sp.audio_features(tracks = batch)
        for i, t in enumerate(track_results['tracks']):
            if t == None:
                none_count += none_count + 1
            else:
                popularity.append(t['popularity'])
                uri.append(t['uri'])
                artists_uri.append(t['artists'][0]['uri'])
    for i, t in enumerate(features_results):
        if t == None:
            none_count2 += none_count2 + 1
        else:
            features.append(t)
        return(features, popularity,uri, artists_uri)

In [ ]:
genres = []
artists_name = []
none_count3 = 0
artists_uri = list(set(artists_uri))

for i in range(0, len(artists_uri), batchsize):
    batch = artists_uri[i:i+batchsize]
    artist_results = sp.artists(batch)
    for i,t in enumerate(artist_results['artists']):
        if t['genres'] == []:
            t['genres'] == np.nan
        else:
            genres.append(t['genres'])
            artists_name.append(t['name'])


In [ ]:
artists_genres = pd.DataFrame(
    {'artist_name':artists_name,
     'genres':genres
    })
print(artists_genres.head(),
artists_genres.info())
genre = artists_genres.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
genre.name = 'genre'
artists_genres = artists_genres.drop('genres', axis=1).join(genre).reset_index(drop=True)


In [ ]:
features = pd.DataFrame.from_dict(features)
features = features.drop(['id','analysis_url','type','track_href','type'], axis=1)
features = features.set_index("uri")
df_pop = pd.DataFrame(
    {'uri': uri,
     'popularity': popularity
    })
df_pop = df_pop.set_index("uri")


In [ ]:
all_features = pd.merge(features,df_pop, left_index=True, right_index=True)
all_features.head()


### Saving files as pickle

In [ ]:
pickle.dump(artists_genres, open("artists_genres.p", 'wb'))

In [ ]:
pickle.dump(all_features,open("all_features.p","wb"))